In [ ]:
!pip install -q datasets accelerate evaluate wandb

In [ ]:
from huggingface_hub import login
login()

In [ ]:
import wandb
wandb.login()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
import torch
torch.cuda.empty_cache()
# Make sure we're using a NVIDIA GPU
import torch
if torch.cuda.is_available():
  gpu_info = !nvidia-smi
  gpu_info = '\n'.join(gpu_info)
  if gpu_info.find("failed") >= 0:
    print("Not connected to a GPU, to leverage the best of PyTorch 2.0, you should connect to a GPU.")

  # Get GPU name
  gpu_name = !nvidia-smi --query-gpu=gpu_name --format=csv
  gpu_name = gpu_name[1]
  GPU_NAME = gpu_name.replace(" ", "_") # remove underscores for easier saving
  print(f'GPU name: {GPU_NAME}')

  # Get GPU capability score
  GPU_SCORE = torch.cuda.get_device_capability()
  print(f"GPU capability score: {GPU_SCORE}")
  if GPU_SCORE >= (8, 0):
    print(f"GPU score higher than or equal to (8, 0), PyTorch 2.x speedup features available.")
  else:
    print(f"GPU score lower than (8, 0), PyTorch 2.x speedup features will be limited (PyTorch 2.x speedups happen most on newer GPUs).")

  # Print GPU info
  print(f"GPU information:\n{gpu_info}")

else:
  print("PyTorch couldn't find a GPU, to leverage the best of PyTorch 2.0, you should connect to a GPU.")
if GPU_SCORE >= (8, 0):
  print(f"[INFO] Using GPU with score: {GPU_SCORE}, enabling TensorFloat32 (TF32) computing (faster on new GPUs)")
  torch.backends.cuda.matmul.allow_tf32 = True
else:
  print(f"[INFO] Using GPU with score: {GPU_SCORE}, TensorFloat32 (TF32) not available, to use it you need a GPU with score >= (8, 0)")
  torch.backends.cuda.matmul.allow_tf32 = False

GPU name: NVIDIA_L4
GPU capability score: (8, 9)
GPU score higher than or equal to (8, 0), PyTorch 2.x speedup features available.
GPU information:
Tue May 21 17:04:27 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA L4                      Off | 00000000:00:03.0 Off |                    0 |
| N/A   57C    P0              30W /  72W |    573MiB / 23034MiB |      0%      Default |
|         

In [ ]:
from datasets import load_dataset
images = load_dataset("date3k2/raw_real_fake_images")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/7444 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1862 [00:00<?, ? examples/s]

In [ ]:
labels = images["train"].features["label"].names
label2id, id2label = dict(), dict()
for i, label in enumerate(labels):
    label2id[label] = str(i)
    id2label[str(i)] = label

In [ ]:
from transformers import AutoImageProcessor

checkpoint = "google/vit-base-patch16-224"
image_processor = AutoImageProcessor.from_pretrained(checkpoint)

preprocessor_config.json:   0%|          | 0.00/160 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/69.7k [00:00<?, ?B/s]

In [ ]:
from torchvision.transforms import RandomResizedCrop, Compose, Normalize, ToTensor

normalize = Normalize(mean=image_processor.image_mean, std=image_processor.image_std)
size = (
    image_processor.size["shortest_edge"]
    if "shortest_edge" in image_processor.size
    else (image_processor.size["height"], image_processor.size["width"])
)
_transforms = Compose([RandomResizedCrop(size), ToTensor(), normalize])

In [ ]:
def transforms(examples):
    examples["pixel_values"] = [_transforms(img.convert("RGB")) for img in examples["image"]]
    del examples["image"]
    return examples

In [ ]:
transformed_images = images.with_transform(transforms)

In [ ]:
from transformers import DefaultDataCollator

data_collator = DefaultDataCollator()

In [ ]:
import evaluate
metrics = evaluate.combine(
    ["accuracy", "f1","recall","precision"], force_prefix=False
)

In [ ]:
import numpy as np

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=1)
    outputs = metrics.compute(predictions=predictions,references=labels)
    return outputs

In [ ]:
from transformers import AutoModelForImageClassification, TrainingArguments, Trainer, EarlyStoppingCallback

model = AutoModelForImageClassification.from_pretrained(
    checkpoint,
    num_labels=len(labels),
    id2label=id2label,
    label2id=label2id,
    ignore_mismatched_sizes=True,
)

model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([2]) in the model instantiated
- classifier.weight: found shape torch.Size([1000, 768]) in the checkpoint and torch.Size([2, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from PIL import Image
Image.MAX_IMAGE_PIXELS = None  # or set to a larger value like 933120000

In [ ]:
training_args = TrainingArguments(
    f"vit-real-fake-cls",
    remove_unused_columns=False,
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=128,
    per_device_eval_batch_size=128,
    num_train_epochs=10,
    warmup_ratio=0.1,
    logging_steps=1,
    weight_decay=0.01,
    load_best_model_at_end=True,
    push_to_hub=True,
    report_to="wandb",
)

trainer = Trainer(
    model = model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=transformed_images["train"],
    eval_dataset=transformed_images["test"],
    tokenizer=image_processor,
    compute_metrics=compute_metrics,
    callbacks = [EarlyStoppingCallback(early_stopping_patience=3)]
)


In [ ]:
%env WANDB_PROJECT=real-fake-classification
%env WANDB_LOG_MODEL=checkpoint

env: WANDB_PROJECT=real-fake-classification
env: WANDB_LOG_MODEL=checkpoint


In [ ]:
trainer.train()
wandb.finish()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


wandb: Currently logged in as: date3k2. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss,Accuracy,F1,Recall,Precision
1,0.175900,0.221197,0.917293,0.922923,0.897760,0.949537
2,0.190300,0.104704,0.962943,0.965858,0.950341,0.981891
3,0.046300,0.082427,0.969925,0.973025,0.983447,0.962822
4,0.001500,0.076346,0.976369,0.978661,0.982473,0.974879
5,0.063100,0.079416,0.973684,0.975850,0.963973,0.988024
6,0.011400,0.058213,0.980129,0.981925,0.978578,0.985294
7,0.000400,0.066161,0.980666,0.982422,0.979552,0.985309
8,0.023100,0.071274,0.975295,0.977340,0.965920,0.989033
9,0.001700,0.051794,0.981740,0.983382,0.979552,0.987242
10,0.026800,0.038491,0.983888,0.985465,0.990263,0.980714


eval/accuracy,▁▆▇▇▇██▇██
eval/f1,▁▆▇▇▇██▇██
eval/loss,█▄▃▂▃▂▂▂▂▁
eval/precision,▁▇▃▅█▇▇██▇
eval/recall,▁▅▇▇▆▇▇▆▇█
eval/runtime,▃█▁▇▇▄█▇▆▄
eval/samples_per_second,▆▁█▂▁▅▁▂▃▅
eval/steps_per_second,▆▁█▂▂▅▁▂▃▅
train/epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
train/global_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
train/grad_norm,█▅▆▄▅▄▄▅▅▇▃▄▄▂▃▄▁▂▂▃▄▃▂▆▁▂▃▃▂▂▃▁▁▄▅▃▂▁▁▁


In [ ]:
trainer.save_model()

In [ ]:
trainer.push_to_hub()

CommitInfo(commit_url='https://huggingface.co/date3k2/vit-real-fake-cls/commit/f6ea28305ad1a6568bb407e18968d7f474fed024', commit_message='End of training', commit_description='', oid='f6ea28305ad1a6568bb407e18968d7f474fed024', pr_url=None, pr_revision=None, pr_num=None)